In [2]:
import json
from PIL import Image

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import clip
from transformers import CLIPProcessor, CLIPModel

/home/kcdh/miniconda3/envs/hanish/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" 

# Load pre-tested CLIP model
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

In [5]:
model.load_state_dict(torch.load("vit_model.pth"))


/tmp/ipykernel_38364/3024784646.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("vit_model.pth"))


<All keys matched successfully>

In [6]:
json_path = '../data/data_test.json'
image_path = '../data/images/test/'

with open(json_path, 'r') as f:
    input_data = []
    for line in f:
        obj = json.loads(line)
        input_data.append(obj)

In [7]:
class image_label_dataset():
    def __init__(self, list_image_path,list_txt):
        self.image_path = list_image_path
        self.label  = clip.tokenize(list_txt)

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx]))
        label = self.label[idx]
        return image, label

In [8]:
list_image_path = []
list_txt = []
for item in input_data:
  img_path = image_path + item['image_path'].split('/')[-1]
  label = item['label']
  list_image_path.append(img_path)
  list_txt.append(label)

In [28]:
test_dataset = image_label_dataset(list_image_path, list_txt)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True) 

In [30]:
test_dataloader

In [15]:
import pandas as pd

df= pd.read_csv("../data/data2.csv")

df_test = df[(df['tag']== 'test')]

In [41]:
image_path = '../' +df_test['image_path'].iloc[0]
image = preprocess(Image.open(image_path))

labeling ={0: 'Erythroplakia',
            1: 'Erythroleukoplakia',
            2: 'Keratosis',
            3: 'Leukoplakia',
            4: 'Tumor',
            5: 'Ulcer'}

true_label = labeling[df_test['label'].iloc[0]]

image_path, true_label

('../data/images/test/45_DSCN6721.jpg', 'Leukoplakia')